In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
import dask.array as da

In [2]:
#preprocessing parts
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense,Dropout,Input,Embedding,LSTM
from keras.models import Sequential,Model
from keras.utils import to_categorical
from keras.initializers import Constant
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix,f1_score,recall_score,precision_score
from keras.layers import Conv1D,MaxPooling1D,GlobalMaxPooling1D
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import inflect
import string
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix,f1_score,recall_score,precision_score,roc_curve,roc_auc_score
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from tqdm import tqdm

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
df1=pd.read_csv("train.csv")
print(df1.shape)
print(df1.columns)

(5693, 6)
Index(['id', 'App Version Code', 'App Version Name', 'Review Text',
       'Review Title', 'Star Rating'],
      dtype='object')


In [156]:
df2=pd.read_csv("test.csv")
print(df2.shape)
print(df2.columns)

(1424, 5)
Index(['id', 'App Version Code', 'App Version Name', 'Review Text',
       'Review Title'],
      dtype='object')


In [7]:
df1.head()

,id,App Version Code,App Version Name,Review Text,Review Title,Star Rating
0,b817b0f0-a2f8-4c9d-a5f6-d3fbf8b1d7e9,NaN,NaN,Very bad wallet balance not use.,NaN,1
1,c7844e8f-56c1-487b-ae3e-df2fdf4c1767,43.0,2.30,Froud app i recharge 199 but not done also sen...,NaN,1
2,93ff57f7-9e02-4fa9-b779-3db82b8af0a4,52.0,2.12,Waste to write comments also,Waste no use,2
3,df2dcdef-c09a-4f35-afab-e1231d3fec9a,62.0,2.21,Nice apo,NaN,5
4,11f8f968-4cec-4424-8427-9709ab05b5be,69.0,2.28,Good nice app,NaN,5


In [8]:
np.sum(df1.isnull())

id                     0
App Version Code    1139
App Version Name    1139
Review Text            1
Review Title        5091
Star Rating            0
dtype: int64

In [158]:
np.sum(df2.isnull())

id                     0
App Version Code     273
App Version Name     273
Review Text            1
Review Title        1244
dtype: int64

In [159]:
df2=df2.dropna(axis=0, subset=['Review Text'])

In [21]:
df1=df1.dropna(axis=0, subset=['Review Text'])

In [22]:
df1['Review Title']=df1['Review Title'].replace(np.nan,df1['Review Text'])

In [23]:
np.sum(df1.isnull())

id                     0
App Version Code    1138
App Version Name    1138
Review Text            0
Review Title           0
Star Rating            0
dtype: int64

In [24]:
maxlen_doc=max(len(docs) for docs in df1['Review Text'])

In [25]:
maxlen_doc

1816

### Text Preprocessing Part

In [26]:
apostropheList = {"n't" : "not","aren't" : "are not","can't" : "cannot","couldn't" : "could not","didn't" : "did not","doesn't" : "does not", \
                  "don't" : "do not","hadn't" : "had not","hasn't" : "has not","haven't" : "have not","he'd" : "he had","he'll" : "he will", \
                  "he's" : "he is","I'd" : "I had","I'll" : "I will","I'm" : "I am","I've" : "I have","isn't" : "is not","it's" : \
                  "it is","let's" : "let us","mustn't" : "must not","shan't" : "shall not","she'd" : "she had","she'll" : "she will", \
                  "she's" : "she is", "shouldn't" : "should not","that's" : "that is","there's" : "there is","they'd" : "they had", \
                  "they'll" : "they will", "they're" : "they are","they've" : "they have","we'd" : "we had","we're" : "we are","we've" : "we have", \
                  "weren't" : "were not", "what'll" : "what will","what're" : "what are","what's" : "what is","what've" : "what have", \
                  "where's" : "where is","who'd" : "who had", "who'll" : "who will","who're" : "who are","who's" : "who is","who've" : "who have", \
                    "won't" : "will not","wouldn't" : "would not", "you'd" : "you had","you'll" : "you will","you're" : "you are","you've" : "you have",\
                    "It's": "It is"}

lemm=WordNetLemmatizer()
p=inflect.engine()
def remove_punctuation(sent):
    #removes punctuation 
    punct=string.punctuation
    l=[]
    for chrs in sent:
        if chrs not in punct:
            l.append(chrs)
    return ''.join(l)

def lower_numbers(sent):
    #lower the words and converts numbers to words like 2-two
    l=[]
    for word in sent.strip().split():
        if word.isdigit():
            try:
                l.append(p.number_to_words(word))
            except:
                l.append('')
        else:
            l.append(word.lower())
    return ' '.join(l)

def remove_stopwords(sent):
    stpwrds=stopwords.words('english')
    l=[word for word in sent.strip().split() if word not in stpwrds]
    return ' '.join(l)

def remove_non_ascii(sent):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in sent.strip().split():
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return ' '.join(new_words)

def lemmatize(sent):
    #lemmatizes the tokens
    words=word_tokenize(sent)
    l=[]
    for word in words:
        l.append(lemm.lemmatize(word))
    return ' '.join(l)

In [27]:
def text_preprocessing(data):
    text_list=[]
    for texts in data:
        try:
            doc=remove_non_ascii(texts)
            doc=remove_punctuation(doc)
            doc=lower_numbers(doc)
            doc=remove_stopwords(doc)
            doc=lemmatize(doc)
        except AttributeError:
            doc=''
        text_list.append(doc)
    return text_list

In [49]:
texts=[]
for doc in df1['Review Text']:
    doc=doc.split()
    text=[]
    for word in doc:
        if word in apostropheList.keys():
            text.append(apostropheList[word])
        else:
            text.append(word)
    #doc=[apostropheList[word] for word in doc if word in apostropheList.keys() else ]
    texts.append(" ".join(text))

In [50]:
texts=text_preprocessing(texts)
df1['cleaned review text']=texts
texts=[]
for doc in df1['Review Title']:
    doc=doc.split()
    text=[]
    for word in doc:
        if word in apostropheList.keys():
            text.append(apostropheList[word])
        else:
            text.append(word)
    #doc=[apostropheList[word] for word in doc if word in apostropheList.keys() else ]
    texts.append(" ".join(text))
texts=text_preprocessing(texts)
df1['cleaned review title']=texts

In [160]:
texts=[]
for doc in df2['Review Text']:
    doc=doc.split()
    text=[]
    for word in doc:
        if word in apostropheList.keys():
            text.append(apostropheList[word])
        else:
            text.append(word)
    #doc=[apostropheList[word] for word in doc if word in apostropheList.keys() else ]
    texts.append(" ".join(text))
texts=text_preprocessing(texts)
df2['cleaned review text']=texts

In [51]:
len_texts=[len(doc) for doc in df1['cleaned review text']]
len_title=[len(doc) for doc in df1['cleaned review title']]
df1['len title']=len_title
df1['len texts']=len_texts

In [163]:
len_texts=[len(doc) for doc in df2['cleaned review text']]
df2['len_text']=len_texts

In [52]:
df1['len title']=df1['len title'].replace(0,np.nan)
df1['len texts']=df1['len texts'].replace(0,np.nan)
print(np.sum(df1.isnull()))

id                         0
App Version Code        1138
App Version Name        1138
Review Text                0
Review Title               0
Star Rating                0
cleaned review text        0
cleaned review title       0
len title                 17
len texts                 18
dtype: int64


In [166]:
df2['len_text']=df2['len_text'].replace(0,np.nan)
print(np.sum(df2.isnull()))

id                        0
App Version Code        272
App Version Name        272
Review Text               0
Review Title           1244
cleaned review text       0
len_texts                 0
len_text                 11
dtype: int64


In [53]:
df1=df1.dropna(axis=0,subset=['len texts'])
print(np.sum(df1.isnull()))

id                         0
App Version Code        1133
App Version Name        1133
Review Text                0
Review Title               0
Star Rating                0
cleaned review text        0
cleaned review title       0
len title                  0
len texts                  0
dtype: int64


In [168]:
df2=df2.dropna(axis=0,subset=['len_text'])
print(np.sum(df2.isnull()))

id                        0
App Version Code        269
App Version Name        269
Review Text               0
Review Title           1234
cleaned review text       0
len_texts                 0
len_text                  0
dtype: int64


In [54]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

In [55]:
df1['Star Rating'].value_counts()

5    2909
1    1786
4     610
3     215
2     154
Name: Star Rating, dtype: int64

In [87]:
train_x,test_x,train_y,test_y=train_test_split(df1['cleaned review text'],df1['Star Rating'],test_size=0.2)

In [57]:
print(train_x.shape)
print(test_x.shape)

(4539,)
(1135,)


### Using Countvectorizer with naive bayes

In [141]:
vec=CountVectorizer(max_features=2000)
train_x1=vec.fit_transform(train_x)
test_x1=vec.transform(test_x)
print(train_x1.shape)
print(test_x1.shape)
clf=MultinomialNB()
clf.fit(train_x1,train_y)
predy_test=clf.predict(test_x1)
print(predy_test.shape)
print("Test set prediction statistics")
print(classification_report(test_y,predy_test))
predy_train=clf.predict(train_x1)
print("Train set prediction statistics")
print(classification_report(train_y,predy_train))

(4456, 2000)
(1114, 2000)
(1114,)
Test set prediction statistics
              precision    recall  f1-score   support

           1       0.80      0.88      0.84       347
           2       0.25      0.03      0.05        36
           3       0.00      0.00      0.00        40
           4       0.26      0.05      0.08       125
           5       0.77      0.95      0.85       566

   micro avg       0.76      0.76      0.76      1114
   macro avg       0.42      0.38      0.36      1114
weighted avg       0.68      0.76      0.70      1114

Train set prediction statistics
              precision    recall  f1-score   support

           1       0.85      0.89      0.87      1428
           2       0.78      0.25      0.38       113
           3       0.71      0.21      0.32       170
           4       0.71      0.15      0.24       476
           5       0.79      0.97      0.87      2269

   micro avg       0.81      0.81      0.81      4456
   macro avg       0.77      0.49 

In [139]:
vec1=CountVectorizer()
train_x2=vec1.fit_transform(train_x)
test_x2=vec1.transform(test_x)
clf2=MultinomialNB()
clf2.fit(train_x2,train_y)
predy_test=clf2.predict(test_x2)
predy_train=clf2.predict(train_x2)
print("Test set prediction statistics")
print(classification_report(test_y,predy_test))
print("Train set prediction statistics")
print(classification_report(train_y,predy_train))

Test set prediction statistics


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           1       0.79      0.92      0.85       347
           2       0.00      0.00      0.00        36
           3       0.00      0.00      0.00        40
           4       0.40      0.02      0.03       125
           5       0.77      0.95      0.85       566

   micro avg       0.77      0.77      0.77      1114
   macro avg       0.39      0.38      0.35      1114
weighted avg       0.68      0.77      0.70      1114

Train set prediction statistics
              precision    recall  f1-score   support

           1       0.84      0.92      0.88      1428
           2       0.94      0.15      0.26       113
           3       0.87      0.15      0.26       170
           4       0.87      0.12      0.22       476
           5       0.79      0.97      0.87      2269

   micro avg       0.81      0.81      0.81      4456
   macro avg       0.86      0.46      0.50      4456
weighted avg       0.82      0.81      0.77  

### tfidfVectorizer with MultinomialNB

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec3=TfidfVectorizer()
train_x3=vec3.fit_transform(train_x)
test_x3=vec3.transform(test_x)
clf3=MultinomialNB()
clf3.fit(train_x3,train_y)
predy_test=clf2.predict(test_x3)
predy_train=clf2.predict(train_x3)
print("Test set prediction statistics")
print(classification_report(test_y,predy_test))
print("Train set prediction statistics")
print(classification_report(train_y,predy_train))

Test set prediction statistics


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           1       0.81      0.87      0.84       356
           2       0.00      0.00      0.00        29
           3       0.00      0.00      0.00        45
           4       0.50      0.01      0.02       108
           5       0.76      0.96      0.85       597

   micro avg       0.78      0.78      0.78      1135
   macro avg       0.41      0.37      0.34      1135
weighted avg       0.70      0.78      0.71      1135

Train set prediction statistics
              precision    recall  f1-score   support

           1       0.85      0.92      0.88      1430
           2       1.00      0.06      0.11       125
           3       1.00      0.06      0.11       170
           4       0.92      0.09      0.16       502
           5       0.77      0.98      0.86      2312

   micro avg       0.80      0.80      0.80      4539
   macro avg       0.91      0.42      0.43      4539
weighted avg       0.83      0.80      0.74  

In [88]:
vec4=TfidfVectorizer(max_features=3000)
train_x4=vec4.fit_transform(train_x)
test_x4=vec4.transform(test_x)
clf4=MultinomialNB()
clf4.fit(train_x4,train_y)
predy_test=clf4.predict(test_x4)
predy_train=clf4.predict(train_x4)
print("Test set prediction statistics")
print(classification_report(test_y,predy_test))
print("Train set prediction statistics")
print(classification_report(train_y,predy_train))

Test set prediction statistics


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           1       0.78      0.92      0.85       347
           2       0.00      0.00      0.00        36
           3       0.00      0.00      0.00        40
           4       0.00      0.00      0.00       125
           5       0.76      0.95      0.85       566

   micro avg       0.77      0.77      0.77      1114
   macro avg       0.31      0.37      0.34      1114
weighted avg       0.63      0.77      0.69      1114

Train set prediction statistics
              precision    recall  f1-score   support

           1       0.82      0.93      0.87      1428
           2       0.00      0.00      0.00       113
           3       0.00      0.00      0.00       170
           4       1.00      0.01      0.02       476
           5       0.78      0.97      0.86      2269

   micro avg       0.79      0.79      0.79      4456
   macro avg       0.52      0.38      0.35      4456
weighted avg       0.77      0.79      0.72  

In [47]:
len(vec3.vocabulary_)

4408

#### tfidf with ANN

In [62]:
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout
from keras.utils.np_utils import to_categorical

In [63]:
def batch_generator(X_data, y_data, batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].toarray()
        #y_batch = y_data[y_data.index[index_batch]]
        y_batch=y_data[index_batch,:]
        counter += 1
        yield X_batch,y_batch
        if (counter > number_of_batches):
            counter=0

In [178]:
def batch_generator2(X_data,batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(X_data.shape[0])
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].toarray()
        #y_batch = y_data[y_data.index[index_batch]]
        #y_batch=y_data[index_batch,:]
        counter += 1
        yield X_batch
        if (counter > number_of_batches):
            counter=0

In [64]:
from keras.callbacks import Callback
from sklearn.metrics import f1_score

class Metrics(Callback):
    def on_train_begin(self,logs={}):
        self.val_f1s=[]
    def on_epoch_end(self,epoch,logs={}):
        pred_y=self.model.predict(self.validation_data[0])
        pred_y=np.argmax(pred_y,axis=1)
        val_tar=np.argmax(self.validation_data[1],axis=1)
        score=f1_score(val_tar,pred_y,average='weighted')
        self.val_f1s.append(score)
        print("val weighted f1: ",score)
        return
metric1=Metrics()

### ANN with tfidfvectorizer

In [65]:
model1=Sequential()
model1.add(Dense(64,activation='relu',input_dim=len(vec3.vocabulary_)))
model1.add(Dropout(0.2))
model1.add(Dense(5,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer='adam')

In [144]:
train_y1=train_y-1
test_y1=test_y-1
train_y_hotenc=to_categorical(train_y1)
test_y_hotenc=to_categorical(test_y1)

In [68]:
model1.fit_generator(generator=batch_generator(train_x3,train_y_hotenc, 32),
                    epochs=5, validation_data=(test_x3, test_y_hotenc),steps_per_epoch=train_x3.shape[0]/32,callbacks=[metric1])

Epoch 1/5
142/141 [==============================] - ETA: 2s - loss: 0.485 - ETA: 2s - loss: 0.404 - ETA: 2s - loss: 0.447 - ETA: 2s - loss: 0.423 - ETA: 2s - loss: 0.437 - ETA: 2s - loss: 0.444 - ETA: 2s - loss: 0.437 - ETA: 2s - loss: 0.436 - ETA: 2s - loss: 0.451 - ETA: 1s - loss: 0.465 - ETA: 1s - loss: 0.465 - ETA: 1s - loss: 0.468 - ETA: 1s - loss: 0.469 - ETA: 1s - loss: 0.475 - ETA: 1s - loss: 0.467 - ETA: 1s - loss: 0.469 - ETA: 1s - loss: 0.468 - ETA: 1s - loss: 0.472 - ETA: 1s - loss: 0.466 - ETA: 1s - loss: 0.465 - ETA: 1s - loss: 0.466 - ETA: 1s - loss: 0.470 - ETA: 0s - loss: 0.468 - ETA: 0s - loss: 0.466 - ETA: 0s - loss: 0.468 - ETA: 0s - loss: 0.469 - ETA: 0s - loss: 0.476 - ETA: 0s - loss: 0.481 - ETA: 0s - loss: 0.483 - ETA: 0s - loss: 0.485 - ETA: 0s - loss: 0.484 - ETA: 0s - loss: 0.484 - ETA: 0s - loss: 0.488 - ETA: 0s - loss: 0.485 - ETA: 0s - loss: 0.484 - ETA: 0s - loss: 0.484 - ETA: 0s - loss: 0.484 - ETA: 0s - loss: 0.485 - ETA: 0s - loss: 0.483 - ETA: 0s - l

C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


val weighted f1:  0.7284149773276202
Epoch 2/5
142/141 [==============================] - ETA: 1s - loss: 0.453 - ETA: 2s - loss: 0.377 - ETA: 2s - loss: 0.408 - ETA: 2s - loss: 0.383 - ETA: 2s - loss: 0.399 - ETA: 2s - loss: 0.406 - ETA: 2s - loss: 0.406 - ETA: 1s - loss: 0.402 - ETA: 1s - loss: 0.415 - ETA: 1s - loss: 0.426 - ETA: 1s - loss: 0.434 - ETA: 1s - loss: 0.424 - ETA: 1s - loss: 0.429 - ETA: 1s - loss: 0.431 - ETA: 1s - loss: 0.435 - ETA: 1s - loss: 0.433 - ETA: 1s - loss: 0.426 - ETA: 1s - loss: 0.431 - ETA: 1s - loss: 0.428 - ETA: 1s - loss: 0.428 - ETA: 1s - loss: 0.426 - ETA: 1s - loss: 0.422 - ETA: 1s - loss: 0.424 - ETA: 1s - loss: 0.424 - ETA: 1s - loss: 0.428 - ETA: 0s - loss: 0.426 - ETA: 0s - loss: 0.428 - ETA: 0s - loss: 0.429 - ETA: 0s - loss: 0.431 - ETA: 0s - loss: 0.439 - ETA: 0s - loss: 0.438 - ETA: 0s - loss: 0.440 - ETA: 0s - loss: 0.440 - ETA: 0s - loss: 0.440 - ETA: 0s - loss: 0.440 - ETA: 0s - loss: 0.443 - ETA: 0s - loss: 0.442 - ETA: 0s - loss: 0.441 

### ANN with countvectorizer

In [149]:
model2=Sequential()
model2.add(Dense(64,activation='relu',input_dim=len(vec1.vocabulary_)))
model2.add(Dropout(0.2))
model2.add(Dense(5,activation='softmax'))
model2.compile(loss='categorical_crossentropy',optimizer='adam')

In [151]:
model2.fit_generator(generator=batch_generator(train_x2,train_y_hotenc, 32),
                    epochs=10, validation_data=(test_x2, test_y_hotenc),steps_per_epoch=train_x2.shape[0]/32,callbacks=[metric1])

Epoch 1/10
140/139 [==============================] - ETA: 0s - loss: 0.493 - ETA: 1s - loss: 0.358 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 0s - loss: 0.399 - ETA: 0s - loss: 0.399 - ETA: 0s - loss: 0.389 - ETA: 0s - loss: 0.403 - ETA: 0s - loss: 0.407 - ETA: 0s - loss: 0.407 - ETA: 0s - loss: 0.414 - ETA: 0s - loss: 0.416 - ETA: 0s - loss: 0.413 - ETA: 0s - loss: 0.409 - ETA: 0s - loss: 0.410 - ETA: 0s - loss: 0.408 - ETA: 0s - loss: 0.405 - 1s 7ms/step - loss: 0.4028 - val_loss: 0.7873
val weighted f1:  0.6975569860340413
Epoch 2/10
140/139 [==============================] - ETA: 2s - loss: 0.386 - ETA: 1s - loss: 0.297 - ETA: 1s - loss: 0.332 - ETA: 0s - loss: 0.345 - ETA: 0s - loss: 0.348 - ETA: 0s - loss: 0.354 - ETA: 0s - loss: 0.352 - ETA: 0s - loss: 0.364 - ETA: 0s - loss: 0.364 - ETA: 0s - loss: 0.373 - ETA: 0s - loss: 0.376 - ETA: 0s - loss: 0.375 - ETA: 0s - loss: 0.372 - ETA: 0s - loss: 0.373 - ETA: 0s - loss: 0.371 - ETA: 0s - loss: 0.369 - 1s 7ms/step - los

### TFIDFVectorizer with LinearSVM

In [89]:
from sklearn.svm import LinearSVC
clf5=LinearSVC(C=0.1)
clf5.fit(train_x4,train_y)
predy_test=clf5.predict(test_x4)
predy_train=clf5.predict(train_x4)
print("Test set prediction statistics")
print(classification_report(test_y,predy_test))
print("Train set prediction statistics")
print(classification_report(train_y,predy_train))

Test set prediction statistics


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           1       0.75      0.95      0.84       347
           2       0.00      0.00      0.00        36
           3       0.00      0.00      0.00        40
           4       0.17      0.01      0.02       125
           5       0.78      0.93      0.85       566

   micro avg       0.77      0.77      0.77      1114
   macro avg       0.34      0.38      0.34      1114
weighted avg       0.65      0.77      0.69      1114

Train set prediction statistics
              precision    recall  f1-score   support

           1       0.81      0.97      0.88      1428
           2       0.00      0.00      0.00       113
           3       1.00      0.01      0.01       170
           4       0.76      0.03      0.06       476
           5       0.80      0.97      0.88      2269

   micro avg       0.81      0.81      0.81      4456
   macro avg       0.67      0.39      0.37      4456
weighted avg       0.79      0.81      0.74  

### Using Word2vec as featurevector and passing it through different classifiers

In [70]:
from gensim.models import Word2Vec
docs=[doc.split() for doc in df1['cleaned review text']]
model_w2v1=Word2Vec(docs,window=5,min_count=3)

In [71]:
word_index={t[0]:idx+1 for idx,t in enumerate(model_w2v1.wv.vocab.items())}
inv_wrd_index={val:key for key,val in word_index.items()}
vocab_len=len(model_w2v1.wv.vocab)
vocab_len=vocab_len+1

In [72]:
emb_mat=np.random.randn(vocab_len,100)
for word,idx in word_index.items():
    emb_mat[idx]=model_w2v1.wv[word]

In [73]:
docs=[]
for doc in df1['cleaned review text']:
    text=[]
    for word in doc.split():
        if word in word_index.keys():
            text.append(word_index[word])
    docs.append(text)

In [74]:
docs=np.array(docs)
df1['tokens cleaned texts']=docs
df1['len tokens']=[len(doc) for doc in df1['tokens cleaned texts']]
df1['len tokens']=df1['len tokens'].replace(0,np.nan)
print(np.sum(df1.isnull()))

id                         0
App Version Code        1133
App Version Name        1133
Review Text                0
Review Title               0
Star Rating                0
cleaned review text        0
cleaned review title       0
len title                  0
len texts                  0
tokens cleaned texts       0
len tokens               104
dtype: int64


In [75]:
df1=df1.dropna(axis=0,subset=['len tokens'])
print(np.sum(df1.isnull()))
print(df1.shape)

id                         0
App Version Code        1099
App Version Name        1099
Review Text                0
Review Title               0
Star Rating                0
cleaned review text        0
cleaned review title       0
len title                  0
len texts                  0
tokens cleaned texts       0
len tokens                 0
dtype: int64
(5570, 12)


In [76]:
from keras.preprocessing.sequence import pad_sequences
maxlen_texts=50
docs=df1['tokens cleaned texts']
docs2_X=pad_sequences(docs,padding='post',maxlen=maxlen_texts)
word_index['UNK']=0
inv_wrd_index[0]='UNK'

In [77]:
embedd_dim=100
embedding_layer_texts=Embedding(vocab_len,embedd_dim,embeddings_initializer=Constant(emb_mat),
                              input_length=maxlen_texts,trainable=False)

In [147]:
from keras.layers import Conv1D,MaxPooling1D,GlobalMaxPooling1D
seq_input=Input(shape=(maxlen_texts,),dtype='int32')
embed_mat=embedding_layer_texts(seq_input)
x=Conv1D(filters=64,kernel_size=5,activation='relu')(embed_mat)
x=MaxPooling1D(pool_size=5,strides=1)(x)
x=GlobalMaxPooling1D()(x)
preds=Dense(5,activation='softmax')(x)
model3=Model(seq_input,preds)
print(model3.summary())
model3.compile(loss='sparse_categorical_crossentropy',optimizer='adam')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 64)                276096    
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 325       
Total params: 276,421
Trainable params: 276,421
Non-trainable params: 0
_________________________________________________________________
None


In [79]:
X=docs2_X
Y=df1['Star Rating']-1
print(X.shape)
print(Y.shape)

(5570, 50)
(5570,)


In [80]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=0.1)
model3.fit(train_x,train_y,validation_data=(test_x,test_y),epochs=10,callbacks=[metric1])

Train on 5013 samples, validate on 557 samples
Epoch 1/10
5013/5013 [==============================] - ETA: 1:07 - loss: 2.265 - ETA: 18s - loss: 1.449 - ETA: 11s - loss: 1.35 - ETA: 8s - loss: 1.2665 - ETA: 7s - loss: 1.236 - ETA: 6s - loss: 1.249 - ETA: 5s - loss: 1.222 - ETA: 5s - loss: 1.202 - ETA: 4s - loss: 1.197 - ETA: 4s - loss: 1.197 - ETA: 4s - loss: 1.183 - ETA: 3s - loss: 1.180 - ETA: 3s - loss: 1.178 - ETA: 3s - loss: 1.168 - ETA: 3s - loss: 1.171 - ETA: 2s - loss: 1.152 - ETA: 2s - loss: 1.145 - ETA: 2s - loss: 1.144 - ETA: 2s - loss: 1.145 - ETA: 2s - loss: 1.134 - ETA: 2s - loss: 1.131 - ETA: 2s - loss: 1.125 - ETA: 2s - loss: 1.125 - ETA: 1s - loss: 1.121 - ETA: 1s - loss: 1.125 - ETA: 1s - loss: 1.128 - ETA: 1s - loss: 1.132 - ETA: 1s - loss: 1.136 - ETA: 1s - loss: 1.137 - ETA: 1s - loss: 1.131 - ETA: 1s - loss: 1.126 - ETA: 1s - loss: 1.126 - ETA: 1s - loss: 1.123 - ETA: 0s - loss: 1.123 - ETA: 0s - loss: 1.119 - ETA: 0s - loss: 1.118 - ETA: 0s - loss: 1.116 - ETA: 

C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


val weighted f1:  0.5880861850443599
Epoch 2/10
5013/5013 [==============================] - ETA: 2s - loss: 1.183 - ETA: 2s - loss: 1.125 - ETA: 2s - loss: 1.087 - ETA: 2s - loss: 1.075 - ETA: 2s - loss: 1.041 - ETA: 2s - loss: 1.064 - ETA: 2s - loss: 1.052 - ETA: 2s - loss: 1.060 - ETA: 2s - loss: 1.066 - ETA: 2s - loss: 1.073 - ETA: 2s - loss: 1.061 - ETA: 2s - loss: 1.059 - ETA: 2s - loss: 1.077 - ETA: 1s - loss: 1.075 - ETA: 1s - loss: 1.073 - ETA: 1s - loss: 1.077 - ETA: 1s - loss: 1.088 - ETA: 1s - loss: 1.087 - ETA: 1s - loss: 1.090 - ETA: 1s - loss: 1.090 - ETA: 1s - loss: 1.095 - ETA: 1s - loss: 1.098 - ETA: 1s - loss: 1.096 - ETA: 1s - loss: 1.092 - ETA: 1s - loss: 1.092 - ETA: 1s - loss: 1.087 - ETA: 1s - loss: 1.080 - ETA: 1s - loss: 1.075 - ETA: 1s - loss: 1.079 - ETA: 0s - loss: 1.082 - ETA: 0s - loss: 1.084 - ETA: 0s - loss: 1.081 - ETA: 0s - loss: 1.086 - ETA: 0s - loss: 1.088 - ETA: 0s - loss: 1.088 - ETA: 0s - loss: 1.085 - ETA: 0s - loss: 1.084 - ETA: 0s - loss: 1.0

5013/5013 [==============================] - ETA: 2s - loss: 1.161 - ETA: 2s - loss: 1.047 - ETA: 2s - loss: 1.035 - ETA: 2s - loss: 1.060 - ETA: 2s - loss: 1.074 - ETA: 2s - loss: 1.062 - ETA: 2s - loss: 1.069 - ETA: 2s - loss: 1.080 - ETA: 2s - loss: 1.075 - ETA: 2s - loss: 1.060 - ETA: 2s - loss: 1.056 - ETA: 2s - loss: 1.060 - ETA: 2s - loss: 1.061 - ETA: 2s - loss: 1.073 - ETA: 1s - loss: 1.072 - ETA: 1s - loss: 1.075 - ETA: 1s - loss: 1.076 - ETA: 1s - loss: 1.074 - ETA: 1s - loss: 1.078 - ETA: 1s - loss: 1.071 - ETA: 1s - loss: 1.072 - ETA: 1s - loss: 1.066 - ETA: 1s - loss: 1.067 - ETA: 1s - loss: 1.068 - ETA: 1s - loss: 1.065 - ETA: 1s - loss: 1.063 - ETA: 1s - loss: 1.068 - ETA: 1s - loss: 1.062 - ETA: 1s - loss: 1.058 - ETA: 0s - loss: 1.054 - ETA: 0s - loss: 1.051 - ETA: 0s - loss: 1.054 - ETA: 0s - loss: 1.050 - ETA: 0s - loss: 1.051 - ETA: 0s - loss: 1.050 - ETA: 0s - loss: 1.050 - ETA: 0s - loss: 1.053 - ETA: 0s - loss: 1.053 - ETA: 0s - loss: 1.057 - ETA: 0s - loss: 1.0

In [81]:
docs=df1['tokens cleaned texts']
docs_X=np.zeros((docs.shape[0],embedd_dim))
for idx,doc in enumerate(docs):
    X=np.zeros((1,embedd_dim))
    for word in doc:
        v=model_w2v1.wv[inv_wrd_index[word]].reshape((1,embedd_dim))
        X+=v
    X/=len(doc)
    docs_X[idx]=X

In [82]:
model4=Sequential()
model4.add(Dense(64,activation='relu',input_dim=embedd_dim))
#model4.add(Dropout(0.2))
model4.add(Dense(5,activation='softmax'))
model4.compile(loss='sparse_categorical_crossentropy',optimizer='adam')

In [84]:
train_x2,test_x2,train_y2,test_y2=train_test_split(docs_X,Y,test_size=0.1)
model4.fit(train_x2,train_y2,validation_data=(test_x2,test_y2),epochs=20,callbacks=[metric1])

Train on 5013 samples, validate on 557 samples
Epoch 1/20
5013/5013 [==============================] - ETA: 55s - loss: 1.61 - ETA: 2s - loss: 1.3717 - ETA: 1s - loss: 1.275 - ETA: 0s - loss: 1.246 - ETA: 0s - loss: 1.238 - ETA: 0s - loss: 1.232 - ETA: 0s - loss: 1.226 - ETA: 0s - loss: 1.230 - ETA: 0s - loss: 1.223 - ETA: 0s - loss: 1.217 - ETA: 0s - loss: 1.212 - 1s 195us/step - loss: 1.2113 - val_loss: 1.1834


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


val weighted f1:  0.0
Epoch 2/20
5013/5013 [==============================] - ETA: 0s - loss: 1.070 - ETA: 0s - loss: 1.176 - ETA: 0s - loss: 1.185 - ETA: 0s - loss: 1.181 - ETA: 0s - loss: 1.165 - ETA: 0s - loss: 1.188 - ETA: 0s - loss: 1.177 - ETA: 0s - loss: 1.182 - ETA: 0s - loss: 1.182 - ETA: 0s - loss: 1.182 - ETA: 0s - loss: 1.181 - ETA: 0s - loss: 1.182 - 1s 124us/step - loss: 1.1796 - val_loss: 1.1687
val weighted f1:  0.0
Epoch 3/20
5013/5013 [==============================] - ETA: 0s - loss: 1.013 - ETA: 0s - loss: 1.126 - ETA: 0s - loss: 1.182 - ETA: 0s - loss: 1.176 - ETA: 0s - loss: 1.183 - ETA: 0s - loss: 1.184 - ETA: 0s - loss: 1.182 - ETA: 0s - loss: 1.181 - ETA: 0s - loss: 1.178 - ETA: 0s - loss: 1.168 - ETA: 0s - loss: 1.164 - ETA: 0s - loss: 1.166 - 1s 124us/step - loss: 1.1651 - val_loss: 1.1672
val weighted f1:  0.017793594306049824
Epoch 4/20
5013/5013 [==============================] - ETA: 0s - loss: 1.091 - ETA: 0s - loss: 1.145 - ETA: 0s - loss: 1.155 - ETA: 

5013/5013 [==============================] - ETA: 0s - loss: 1.141 - ETA: 0s - loss: 0.956 - ETA: 0s - loss: 0.983 - ETA: 0s - loss: 1.012 - ETA: 0s - loss: 0.999 - ETA: 0s - loss: 1.012 - ETA: 0s - loss: 1.022 - ETA: 0s - loss: 1.016 - ETA: 0s - loss: 1.021 - ETA: 0s - loss: 1.025 - ETA: 0s - loss: 1.022 - ETA: 0s - loss: 1.019 - ETA: 0s - loss: 1.021 - 1s 136us/step - loss: 1.0244 - val_loss: 1.0421
val weighted f1:  0.45706371191135736


#### So after applying all these different methods on the imbalanced dataset, we can conclude that the traditional ML algorithms like naive bayes,Linear SVM are working better than CNNs,LSTMS etc. The highest weighted F1 score obtained is for ANNs using tfidf vectors i.e 72.5%.Lets try it on balanced dataset.

In [103]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

vec5=TfidfVectorizer()
train_x5=vec.fit_transform(train_x)
test_x5=vec.transform(test_x)
print(train_x5.shape)
print(test_x5.shape)
lr=LogisticRegression()
lr.fit(train_x5,train_y)
predy_test=lr.predict(test_x5)
print(predy_test.shape)
print("Test set prediction statistics")
print(classification_report(test_y,predy_test))
predy_train=lr.predict(train_x5)
print("Train set prediction statistics")
print(classification_report(train_y,predy_train))

(4456, 2000)
(1114, 2000)


C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


(1114,)
Test set prediction statistics
              precision    recall  f1-score   support

           1       0.79      0.87      0.83       347
           2       0.00      0.00      0.00        36
           3       0.00      0.00      0.00        40
           4       0.25      0.04      0.07       125
           5       0.76      0.94      0.84       566

   micro avg       0.75      0.75      0.75      1114
   macro avg       0.36      0.37      0.35      1114
weighted avg       0.66      0.75      0.69      1114

Train set prediction statistics
              precision    recall  f1-score   support

           1       0.90      0.95      0.93      1428
           2       0.98      0.42      0.59       113
           3       0.95      0.34      0.50       170
           4       0.94      0.22      0.35       476
           5       0.82      0.99      0.90      2269

   micro avg       0.85      0.85      0.85      4456
   macro avg       0.92      0.58      0.65      4456
weight

In [100]:
np.sum(df1.isnull())

id                         0
App Version Code        1099
App Version Name        1099
Review Text                0
Review Title               0
Star Rating                0
cleaned review text        0
cleaned review title       0
len title                  0
len texts                  0
tokens cleaned texts       0
len tokens                 0
dtype: int64

In [105]:
from imblearn.over_sampling import SMOTE

In [123]:
sm=SMOTE('not majority')
train_x_sm,train_y_sm=sm.fit_sample(train_x5,train_y)
print(train_x_sm.shape)
print(test_x5.shape)
lr=LogisticRegression()
lr.fit(train_x_sm,train_y_sm)
predy_test=lr.predict(test_x5)
print(predy_test.shape)
print("Test set prediction statistics")
print(classification_report(test_y,predy_test))
predy_train=lr.predict(train_x_sm)
print("Train set prediction statistics")
print(classification_report(train_y_sm,predy_train))

(11345, 2000)
(1114, 2000)


C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


(1114,)
Test set prediction statistics
              precision    recall  f1-score   support

           1       0.79      0.80      0.79       347
           2       0.05      0.11      0.07        36
           3       0.11      0.20      0.14        40
           4       0.20      0.41      0.27       125
           5       0.85      0.54      0.66       566

   micro avg       0.58      0.58      0.58      1114
   macro avg       0.40      0.41      0.39      1114
weighted avg       0.70      0.58      0.62      1114

Train set prediction statistics
              precision    recall  f1-score   support

           1       0.89      0.79      0.84      2269
           2       0.57      0.84      0.68      2269
           3       0.52      0.35      0.42      2269
           4       0.50      0.61      0.55      2269
           5       0.81      0.63      0.71      2269

   micro avg       0.64      0.64      0.64     11345
   macro avg       0.66      0.64      0.64     11345
weight

In [125]:
vec3=TfidfVectorizer()
train_x3=vec3.fit_transform(train_x)
test_x3=vec3.transform(test_x)
sm=SMOTE('not majority')
train_x_sm,train_y_sm=sm.fit_sample(train_x3,train_y)
clf3=MultinomialNB()
clf3.fit(train_x_sm,train_y_sm)
predy_test=clf3.predict(test_x3)
predy_train=clf3.predict(train_x_sm)
print("Test set prediction statistics")
print(classification_report(test_y,predy_test))
print("Train set prediction statistics")
print(classification_report(train_y_sm,predy_train))

Test set prediction statistics
              precision    recall  f1-score   support

           1       0.82      0.78      0.80       347
           2       0.07      0.14      0.09        36
           3       0.08      0.17      0.11        40
           4       0.18      0.37      0.24       125
           5       0.86      0.56      0.68       566

   micro avg       0.58      0.58      0.58      1114
   macro avg       0.40      0.40      0.38      1114
weighted avg       0.72      0.58      0.63      1114

Train set prediction statistics
              precision    recall  f1-score   support

           1       0.90      0.87      0.89      2269
           2       0.88      0.94      0.91      2269
           3       0.85      0.77      0.81      2269
           4       0.61      0.70      0.65      2269
           5       0.71      0.65      0.68      2269

   micro avg       0.79      0.79      0.79     11345
   macro avg       0.79      0.79      0.79     11345
weighted avg  

In [127]:
vec1=CountVectorizer()
train_x2=vec1.fit_transform(train_x)
test_x2=vec1.transform(test_x)
sm=SMOTE('not majority')
train_x_sm,train_y_sm=sm.fit_sample(train_x2,train_y)
clf2=MultinomialNB()
clf2.fit(train_x_sm,train_y_sm)
predy_test=clf2.predict(test_x2)
predy_train=clf2.predict(train_x_sm)
print("Test set prediction statistics")
print(classification_report(test_y,predy_test))
print("Train set prediction statistics")
print(classification_report(train_y_sm,predy_train))

Test set prediction statistics
              precision    recall  f1-score   support

           1       0.79      0.88      0.83       347
           2       0.06      0.06      0.06        36
           3       0.10      0.10      0.10        40
           4       0.18      0.37      0.24       125
           5       0.82      0.58      0.68       566

   micro avg       0.62      0.62      0.62      1114
   macro avg       0.39      0.40      0.38      1114
weighted avg       0.69      0.62      0.64      1114

Train set prediction statistics
              precision    recall  f1-score   support

           1       0.41      0.84      0.56      2269
           2       0.70      0.28      0.40      2269
           3       0.67      0.18      0.28      2269
           4       0.45      0.67      0.54      2269
           5       0.76      0.63      0.69      2269

   micro avg       0.52      0.52      0.52     11345
   macro avg       0.60      0.52      0.49     11345
weighted avg  

In [152]:
model5=Sequential()
model5.add(Dense(64,activation='relu',input_dim=len(vec1.vocabulary_)))
model5.add(Dropout(0.2))
model5.add(Dense(5,activation='softmax'))
model5.compile(loss='categorical_crossentropy',optimizer='adam')

In [153]:
train_y1=train_y_sm-1
test_y1=test_y-1
train_y_hotenc=to_categorical(train_y1)
test_y_hotenc=to_categorical(test_y1)
model5.fit_generator(generator=batch_generator(train_x_sm,train_y_hotenc, 32),
                    epochs=5, validation_data=(test_x2, test_y_hotenc),steps_per_epoch=train_x3.shape[0]/32,callbacks=[metric1])

Epoch 1/5
140/139 [==============================] - ETA: 34s - loss: 1.62 - ETA: 4s - loss: 1.5880 - ETA: 2s - loss: 1.539 - ETA: 1s - loss: 1.499 - ETA: 1s - loss: 1.461 - ETA: 1s - loss: 1.412 - ETA: 0s - loss: 1.368 - ETA: 0s - loss: 1.342 - ETA: 0s - loss: 1.297 - ETA: 0s - loss: 1.267 - ETA: 0s - loss: 1.244 - ETA: 0s - loss: 1.213 - ETA: 0s - loss: 1.185 - ETA: 0s - loss: 1.159 - ETA: 0s - loss: 1.129 - ETA: 0s - loss: 1.121 - 1s 9ms/step - loss: 1.1170 - val_loss: 0.8414


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


val weighted f1:  0.6848157678189807
Epoch 2/5
140/139 [==============================] - ETA: 0s - loss: 1.121 - ETA: 0s - loss: 1.022 - ETA: 0s - loss: 0.879 - ETA: 0s - loss: 0.733 - ETA: 0s - loss: 1.174 - ETA: 0s - loss: 1.449 - ETA: 0s - loss: 1.548 - ETA: 0s - loss: 1.575 - ETA: 0s - loss: 1.565 - ETA: 0s - loss: 1.547 - ETA: 0s - loss: 1.524 - ETA: 0s - loss: 1.494 - ETA: 0s - loss: 1.581 - ETA: 0s - loss: 1.623 - ETA: 0s - loss: 1.644 - ETA: 0s - loss: 1.650 - ETA: 0s - loss: 1.647 - ETA: 0s - loss: 1.636 - 1s 8ms/step - loss: 1.6345 - val_loss: 1.4625
val weighted f1:  0.4402413851590232
Epoch 3/5
140/139 [==============================] - ETA: 0s - loss: 1.331 - ETA: 0s - loss: 1.231 - ETA: 0s - loss: 1.150 - ETA: 0s - loss: 1.596 - ETA: 0s - loss: 1.775 - ETA: 0s - loss: 1.839 - ETA: 0s - loss: 1.804 - ETA: 0s - loss: 1.747 - ETA: 0s - loss: 1.669 - ETA: 0s - loss: 1.653 - ETA: 0s - loss: 1.614 - ETA: 0s - loss: 1.565 - ETA: 0s - loss: 1.520 - ETA: 0s - loss: 1.463 - ETA: 0

#### After going through all different methods we can conclude that ANNs with TfidfVectorizer is giving the best weighted f1 score.That is we will be using model1.

In [214]:
test_final_x4=vec3.transform(df2['cleaned review text'])
pred_test_final=model1.predict_generator(generator=batch_generator2(test_final_x4,32),steps=test_final_x4.shape[0]/32)
pred_test_final=np.argmax(pred_test_final,axis=1)
print(pred_test_final.shape)
pred_test_final=pred_test_final+1
df2['Star Rating']=pred_test_final

(1412,)


In [215]:
df3=pd.read_csv("test.csv")
df3['Star Rating']=5*np.ones((df3.shape[0],1),dtype='int')
for idx in range(df3.shape[0]):
    id1=df3.ix[idx,'id']
    v=df2[df2['id']==id1]['Star Rating']
    if (len(v)!=0):
        df3.ix[idx,'Star Rating']=v.values[0]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [216]:
sub_df=df3[['id','Star Rating']]
sub_df.to_csv("Submission1.csv")